# 6 · Estatísticas Espectrais e Regimes de Escala

**Registro observacional associado ao livro**  
*Descobrindo o Caos nos Números — Como a ordem emerge quando mudamos a forma de observar*  
© Alvaro Costa, 2025  

Este notebook faz parte de uma sequência canônica de registros computacionais.  
Ele não introduz hipóteses, conjecturas ou modelos interpretativos novos.

Seu objetivo é exclusivamente **registrar** o comportamento de estruturas aritméticas sob um regime  
de observação explícito, determinístico e reproduzível.

A leitura conceitual completa encontra-se no livro. Este notebook documenta apenas o experimento  
correspondente.

**Licença:** Creative Commons BY–NC–ND 4.0  
É permitida a leitura, execução e citação.  
Não é permitida a modificação, redistribuição adaptada ou uso comercial independente.

---

## 1. Introdução operacional

Nos capítulos anteriores, construímos o operador determinístico $M$ e analisamos sua estrutura visual. Agora, o foco desloca-se  
da inspeção estética para a **análise estatística**.

O objetivo deste laboratório é registrar como as relações entre os autovalores de $M$ se comportam à medida que o **regime de  
observação** é alterado. Mostraremos que a ordem observada não depende apenas do objeto aritmético subjacente, mas da forma como  
a escala entra na construção do operador.

Nenhuma hipótese de universalidade é assumida neste ponto. As ferramentas são introduzidas para que o leitor **observe o fenômeno**  
antes de qualquer interpretação teórica.

---

## 2. A medida do espaçamento

A principal ferramenta deste capítulo é a **Distribuição dos Espaçamentos Consecutivos**, denotada por $P(s)$.

Sejam $\{\lambda_i\}_{i=1}^N$ os autovalores reais do operador $M$, ordenados de forma crescente,

$
\lambda_1 \le \lambda_2 \le \cdots \le \lambda_N .
$

Definimos o espaçamento entre autovalores consecutivos como

$
s_i = \lambda_{i+1} - \lambda_i .
$

Esses espaçamentos não são analisados em valor absoluto. Antes da análise estatística, é aplicada uma normalização da densidade  
espectral, processo conhecido como *unfolding*, cujo objetivo é remover a variação da densidade média de estados e permitir  
a comparação entre espectros distintos.

Neste notebook, o *unfolding* é implementado de duas formas, dependendo do regime de observação:

- de forma **local** na escala linear, para compensar variações residuais da densidade;
- de forma **global** na escala logarítmica, onde a densidade média já se encontra aproximadamente estabilizada.

Após essa normalização, o histograma dos espaçamentos $s_i$ define empiricamente a distribuição $P(s)$, que será utilizada como  
instrumento observacional ao longo do experimento.

### O que observar

- **Regime de ruído (Poisson):**  
  Os autovalores comportam-se de forma aproximadamente independente.  
  A probabilidade de espaçamentos muito pequenos é máxima.

- **Regime correlacionado (repulsão):**  
  Os autovalores evitam-se de forma organizada.  
  A probabilidade de espaçamento nulo é fortemente suprimida,  
  produzindo uma estrutura característica de correlação.

---

## 3. Protocolo experimental: o impacto da escala

O *widget* abaixo permite explorar, de forma controlada, o efeito da **escala de observação** sobre a estatística espectral do operador.

Três parâmetros podem ser ajustados:

- **$N$** — número de pontos amostrados e, consequentemente, a dimensão do operador;
- **$X_0$** — ponto central da região aritmética analisada;
- **`span`** — extensão da janela de observação na escala logarítmica.

Para cada escolha desses parâmetros, o experimento constrói um **único operador espectral**, avaliado sobre **dois conjuntos distintos  
de pontos de amostragem**, correspondentes a **duas métricas de observação** aplicadas à mesma região da reta numérica:

1. **Escala linear**  
   Os pontos de amostragem são distribuídos de forma uniforme em $x$, correspondendo à observação direta da reta numérica aditiva.

2. **Escala logarítmica**  
   Os pontos são distribuídos de forma uniforme em $log x$, alinhando a observação com a estrutura multiplicativa implícita no operador.

Os resultados são apresentados **lado a lado**, permitindo uma comparação visual direta entre os regimes estatísticos emergentes.

Os gráficos exibidos são **registros observacionais**. Nenhuma interpretação teórica é imposta neste estágio.

O objetivo é permitir que o leitor **observe diretamente** como a estatística dos espaçamentos espectrais responde à mudança da régua de  
observação, mantendo fixos o operador e a região aritmética analisada.

> Embora o operador seja intrinsecamente logarítmico, a observação em escala linear é mantida como experimento de contraste, permitindo
> verificar como a perda de coerência de fase destrói a correlação espectral.


In [1]:
# ==========================================================
# NOTEBOOK 06 — TRANSIÇÃO DE ESCALA E ESTRUTURA ESPECTRAL
#
# Operador:
#   M_ij = cos(Δπ(x_i) ln x_j) + cos(Δπ(x_j) ln x_i)
#
# Diferença entre os modos:
#   - Logarítmico: janela multiplicativa
#   - Linear:      mesma janela, observada aditivamente
#
# Requisitos: numpy, matplotlib, ipywidgets
# ==========================================================

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# --- 1. GERAÇÃO DE PRIMOS ---
def generate_primes_upto(n):
    if n < 2:
        return np.array([], dtype=np.int64)
    size = (n - 1) // 2
    sieve = np.ones(size, dtype=bool)
    for i in range(int(np.sqrt(n)) // 2):
        if sieve[i]:
            p = 2*i + 3
            sieve[(p*p-3)//2::p] = False
    return np.concatenate(([2], 2*np.where(sieve)[0] + 3))

# --- 2. FUNÇÕES DE CÁLCULO ---
def get_delta_pi(x_points, primes):
    x_int = np.floor(x_points).astype(int)
    pi_x = np.searchsorted(primes, x_int, side='right')
    pi_x_div_2 = np.searchsorted(primes, x_int // 2, side='right')
    return pi_x - 2 * pi_x_div_2

def generate_matrix(fx, x_values):
    logx = np.log(x_values.astype(np.float64))
    C = np.cos(np.outer(fx.astype(np.float64), logx))
    M = C + C.T
    M = (M - M.mean()) / M.std()
    return 0.5 * (M + M.T)

def normalize_spacings(lam, local=False):
    lam_bulk = np.sort(lam)[int(0.1*len(lam)):int(0.9*len(lam))]
    s = np.diff(lam_bulk)
    s = s[s > 0]
    if local:
        w = 21
        pad = w // 2
        s_padded = np.pad(s, (pad, pad), mode='reflect')
        local_mean = np.convolve(s_padded, np.ones(w)/w, mode='valid')
        return s / local_mean
    return s / s.mean()

# --- 3. FUNÇÃO INTERATIVA ---
def plot_observational(N=2048, log_X0=7, span=2.4):

    X0 = int(10**log_X0)
    max_x = int(X0 * np.exp(span/2)) + N
    primes = generate_primes_upto(max_x)

    # Escala Linear
    x_lin = np.arange(X0, X0 + N)
    s_lin = normalize_spacings(
        np.linalg.eigh(
            generate_matrix(get_delta_pi(x_lin, primes), x_lin)
        )[0],
        local=True
    )

    # Escala Logarítmica
    x_log = np.exp(np.linspace(np.log(X0) - span/2,
                               np.log(X0) + span/2, N))
    s_log = normalize_spacings(
        np.linalg.eigh(
            generate_matrix(get_delta_pi(x_log, primes), x_log)
        )[0],
        local=False
    )

    # Curvas auxiliares (sem nome teórico)
    s_grid = np.linspace(0, 4, 400)
    curve_A = (np.pi * s_grid / 2) * np.exp(-np.pi * s_grid**2 / 4)
    curve_B = np.exp(-s_grid)

    # --- PLOTAGEM ---
    fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

    configs = [
        (s_lin, "a) Escala Linear"),
        (s_log, "b) Escala Logarítmica")
    ]

    for ax, (data, title) in zip(axes, configs):
        ax.hist(
            data[data <= 4],
            bins=50,
            density=True,
            alpha=0.85,
            histtype='stepfilled',
            edgecolor='black',
            linewidth=0.3,
            label='Distribuição observada'
        )
        ax.plot(s_grid, curve_A, 'r--', lw=2.2, label='Curva A')
        ax.plot(s_grid, curve_B, 'k:', lw=2.2, label='Curva B')
        ax.set_title(title, fontsize=14)
        ax.set_xlim(0, 4)
        ax.set_xlabel('s')
        ax.legend()

    fig.suptitle(f"N = {N},  X₀ = 10^{log_X0}", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# --- 4. WIDGETS ---
interact(
    plot_observational,
    N=widgets.Dropdown(
        options=[512, 1024, 2048],
        value=2048,
        description='N'
    ),
    log_X0=widgets.IntSlider(
        min=3, max=8, step=1,
        value=7,
        description='X₀ = 10^',
        continuous_update=False
    ),
    span=widgets.FloatSlider(
        min=1.0, max=4.0, step=0.1,
        value=2.4,
        description='Span (log)',
        continuous_update=False
    )
);


interactive(children=(Dropdown(description='N', index=2, options=(512, 1024, 2048), value=2048), IntSlider(val…

---

## 4. Transparência metodológica: o mecanismo de cálculo

Para garantir a reprodutibilidade dos resultados, o código opera sob um protocolo estrito, dividido em quatro estágios fundamentais:

1. **Geração e amostragem**  
   Um crivo otimizado é utilizado para identificar números primos e calcular o sinal aritmético $\Delta_\pi(x)$. A amostragem é definida  
   pelo regime de escala: no modo **linear**, os pontos são contíguos; no modo **logarítmico**, os pontos são distribuídos exponencialmente  
   para se alinhar a densidade média dos primos.

2. **Construção do operador**  
   O operador é construído como uma matriz de correlação harmônica, na qual cada entrada corresponde à projeção do sinal aritmético sobre  
   uma frequência logarítmica. A matriz é forçada à simetria ($M = M^T$), garantindo um espectro real bem definido.

3. **Filtragem espectral (*bulk analysis*)**  
   Apenas os 90% centrais do espectro são considerados. As regiões de borda, sujeitas a efeitos de truncamento numérico, são descartadas  
   para evitar artefatos estatísticos.

4. **Unfolding (normalização)**  
   Os espaçamentos são normalizados pela densidade espectral média. Esse procedimento não cria correlação nem induz universalidade; ele apenas  
   remove efeitos de escala que poderiam mascarar o regime estatístico já codificado no operador.

> **O unfolding não cria a GOE — ele apenas impede que a densidade média a esconda.**

---

## 5. Parâmetros do protocolo de observação

O protocolo experimental deste notebook é deliberadamente simples. A construção do operador é mantida fixa, e apenas o **regime de observação**  
é modificado.

Não são introduzidos parâmetros de regularização estatística, nem técnicas avançadas de unfolding ou filtragem adaptativa. O objetivo é tornar o  
fenômeno visível antes de qualquer quantificação refinada.

### O parâmetro `span`

O parâmetro `span` controla a extensão da janela de observação na escala logarítmica. Ele define a razão multiplicativa entre os valores mínimo  
e máximo de $x$ incluídos na análise, determinando a largura efetiva da região aritmética examinada.

Essa escolha é traduzida, no código, em uma janela física comum aos dois modos de observação, garantindo que ambos analisem **a mesma região da  
reta numérica**, ainda que sob métricas diferentes.

O valor de `span` é intencionalmente amplo neste notebook, de modo a permitir que eventuais correlações estruturais se manifestem de forma clara,  
caso existam.

### Ausência de `jitter` e parâmetros adicionais

Não é utilizado qualquer parâmetro de perturbação (*jitter*), nem técnicas de suavização local ou reamostragem estatística.

A grade de amostragem é determinística, e a normalização aplicada aos espaçamentos é mínima. Essa escolha evita a introdução prematura de hipóteses  
técnicas e preserva a leitura direta do efeito da escala sobre o espectro.

Métricas escalares mais refinadas, como $\langle r \rangle$ ou $\Sigma^2(L)$, são deliberadamente deixadas para etapas posteriores do trabalho.

---

## 6. Observação metodológica — dependência da régua de observação

Este notebook implementa um **protocolo observacional elementar**, no qual o único grau de liberdade relevante é a escala de observação.

O operador é mantido fixo, e nenhuma hipótese de universalidade é assumida ou testada.

O operador espectral utilizado é construído a partir da projeção

$
M_{ij} =
\cos\!\bigl(\Delta\pi(x_i)\,\log x_j\bigr)
+
\cos\!\bigl(\Delta\pi(x_j)\,\log x_i\bigr),
$

ou seja, sua estrutura interna é explicitamente **logaritmizada** em relação à variável de observação $x$.

Essa escolha não é neutra.

Como consequência direta, o operador entra em ressonância estrutural apenas quando os pontos de amostragem são distribuídos de forma coerente com  
a escala logarítmica. Quando a observação é realizada em escala linear, a coerência de fase é quebrada, e o espectro perde correlação de longo alcance.

Em síntese:

- o operador **não força** estatística correlacionada;
- a estatística correlacionada **emerge apenas** quando a régua de observação é compatível com a estrutura multiplicativa implícita no operador.

A distinção observada entre regimes não deve ser interpretada como uma mudança ontológica do sistema, mas como consequência direta da **adequação  
(ou inadequação) da régua de observação à estrutura do operador**.

> A universalidade estatística não é uma propriedade absoluta do operador, mas da **relação entre operador e observação**.
